In [22]:
import numpy as np
import random as rd
import time as t
import itertools as it


# create a class for board
class nd_sudoku_Board:
    def __init__(self, len_dim: int, num_dim: int):
        self.len_dim = len_dim
        self.num_dim = num_dim
        self.board: np.ndarray = np.zeros([len_dim] * num_dim, dtype=int)
        self.history = []

    def sanity_check(self):
        assert self.num_dim > 1, "The number of dimensions must be greater than 1, instead it is: " + str(self.num_dim)
        assert self.len_dim > 1, "The length of each dimension must be greater than 1, instead it is: " + str(self.len_dim)
        assert type(self.board) == np.ndarray, "The board must be a numpy array, instead it is of type: " + str(type(self.board))
        assert self.board.shape == (self.len_dim,) * self.num_dim, "The board must be of the correct shape, instead it is of shape: " + str(self.board.shape)
        assert self.board.dtype == int, "The board must be an array of integers, instead it is of type: " + str(self.board.dtype)

    def __str__(self):
        return str(self.board)
    
    def copy(self):
        new_board = nd_sudoku_Board(self.len_dim, self.num_dim)
        new_board.board = self.board.copy()
        new_board.history = self.history.copy()
        return new_board

    def get_board(self):
        return self.board
    
    def get_empty_cells(self) -> list[tuple]:
        return [tuple(i) for i in np.argwhere(self.board == 0)]
    
    def get_history(self):
        return self.history


    def set_cell(self, coord: tuple, value: int) -> None:
        self.board[coord] = value
        self.history.append((coord, value))


    def undo_last_move(self) -> None:
        assert len(self.history) > 0, "There are no moves to undo"
        last_coord, _ = self.history.pop()
        self.board[last_coord] = 0
        

    def is_board_full(self):
        return np.all(self.board != 0)
    
    def these_entries_cant_be_in(self, coord: tuple) -> set[int]:
        # get the neighbouring entries of a cell
        # if a cell is in a same row/column/... as another cell, they are neighbours

        # create a list of all the coordinates with a single value changed
        neighbour_coords = []
        for i in range(self.num_dim):
            for j in range(self.len_dim):
                if j != coord[i]:
                    new_coord = list(coord)
                    new_coord[i] = j
                    neighbour_coords.append(tuple(new_coord))

        # get the values of the neighbouring cells
        neighbours = set(self.board[tuple(zip(*neighbour_coords))])
        return neighbours
    
    def is_line_valid(self, line: np.ndarray):
        # check if the line has no duplicates other than 0
        return len(line) - (line == 0).sum() == len(set(line)) - (0 in line)
    
    def is_valid(self):
        board_copy = self.board.copy()
        rotated_board_list = []
        for _ in range(self.num_dim):
            rotated_board_list.append(board_copy.copy())
            board_copy = np.moveaxis(board_copy, 0, -1)
        
        for row, board in zip(it.product(range(self.len_dim), repeat=self.num_dim-1), rotated_board_list):
            if not self.is_line_valid(board[row]):
                return False
        return True
    
    def analyse_history(self):
        # Called when the game is over with the board not solvable
        # Go through the history in the reverse order and find the first move that made the board unsolvable
        # Return the history with solvability appended to each move
        board = self.copy()
        for index, _ in enumerate(reversed(self.history)):
            board.undo_last_move()
            if Board_solver(board)[0]:
                last_valid_index = len(self.history) - index - 2
                break
        
        history_with_solvability = []
        for index, (coord, value) in enumerate(self.history):
            if index <= last_valid_index:
                history_with_solvability.append((coord, value, True))
            else:
                history_with_solvability.append((coord, value, False))
                
        return history_with_solvability
    

def recursive_solver(board: nd_sudoku_Board) -> tuple[bool, nd_sudoku_Board, str]:
    board = board.copy()
    empty_cells = board.get_empty_cells()
    if len(empty_cells) == 0:
        return True, board, ''
    
    coord = empty_cells[0]
    for value in range(1, board.len_dim+1):
        if value not in board.these_entries_cant_be_in(coord):
            board.set_cell(coord, value)
            solved, new_board, reason = recursive_solver(board)
            if solved:
                return True, new_board, ''
            else:
                board.undo_last_move()
                
    return False, board, 'Ran out of options'


def Board_solver(board: nd_sudoku_Board) -> tuple[bool, nd_sudoku_Board, str]:
    if not isinstance(board, nd_sudoku_Board):
        return False, board, 'The board is not an instance of the nd_sudoku_Board class'
    
    if len(board.history) <= 1:
        return True, board, ''
    
    if not board.is_valid():
        return False, board, 'The setup is invalid'

    board = board.copy()

    if board.is_board_full():
        return True, board, ''
    
    return recursive_solver(board)

    
class NDSudokuGame:
    def __init__(self):
        # initialise the game
        self.create_new_board()
        self.main()

    def clear_the_board(self):
        # clear the board
        self.B = nd_sudoku_Board(self.B.len_dim, self.B.num_dim)

    def create_new_board(self):
        # reset the board
        # Ask the user for the number of dimensions and the length of each dimension through the input handler
        # create a board with the given number of dimensions and length of each dimension
        len_dim = self.input_handler_int("Enter the length of each dimension: ", it.count())
        num_dim = self.input_handler_int("Enter the number of dimensions: ", it.count())
        self.B = nd_sudoku_Board(int(len_dim), int(num_dim))


    def play(self):
        # play the game
        # while the board is not full
        #   get the empty cells
        #   randomly select a cell
        #   ask the user for a value for the cell
        #   set the cell to the value
        #   ask the user if the opponent challenges the move
        #   if the opponent challenges the move
        #       check if the board is solvable
        #       if the board is solvable, print the number of solutions
        #       if the board is not solvable, print that the board is not solvable
        #       break
        #   else
        #       continue
        while not self.B.is_board_full():
            empty_cells = self.B.get_empty_cells()
            coord = rd.choice(empty_cells)
            coord_1_indexed = tuple([x+1 for x in coord])
            t.sleep(0.1)

            value = self.input_handler_int( f"Enter the value for {coord_1_indexed}: ", list(range(1, self.B.len_dim+1)) )
            self.B.set_cell(coord, value)
            print(coord_1_indexed, ': ', value)
            t.sleep(0.5)

            Challenge = self.input_handler_yn("Does the opponent challenge the move? (y/n): ")
            if Challenge == "y":
                solvable, example, reason = Board_solver(self.B)
                if solvable:
                    print("The board is solvable!")
                    print("Here is an example solution:")
                    print(example.get_board())
                else:
                    print("The board is not solvable!")
                    print(reason)

                    Analyse = self.input_handler_yn("Do you want to analyse the game? (y/n): ")
                    if Analyse == "y":
                        history_with_solvability = self.B.analyse_history()
                        for coord, value, solvable in history_with_solvability:
                            coord_1_indexed = tuple([x+1 for x in coord])
                            print(coord_1_indexed, ': ', value, ' - ', solvable)
                        print()
                        t.sleep(0.1)

                print()
                t.sleep(0.1)
                break
            else:
                print()
                t.sleep(0.1)
                continue

        print("Game over!")
        t.sleep(0.1)


    def input_handler_yn(self, query: str) -> str:
        # input handler for yes or no questions
        # ask the user for input
        # if the input is yes or no or y or n
        #   return the input
        # else
        #   simply ask the user for input again after printing the error message
        
        while True:
            try:
                Input = input(query)
                if Input in ['stop', 'exit', 'quit', '']:
                    raise KeyboardInterrupt
                assert Input in ["y", "n"], "The input must be either 'y' or 'n'"
                return Input
            except AssertionError as error:
                print(error)
                print("Try again")
                print()
                t.sleep(0.1)
                continue

    
    def input_handler_int(self, query: str, accepted_list) -> int:
        # input handler for integer inputs
        # ask the user for input
        # if the input is an integer
        #   return the input
        # else
        #   simply ask the user for input again after printing the error message
        
        while True:
            try:
                Input = input(query)
                if Input in ['stop', 'exit', 'quit', '']:
                    raise KeyboardInterrupt
                Input = int(Input)
                if Input in accepted_list:
                    return Input
            except ValueError:
                print("The input must be an integer")
                print("Try again")
                print()
                t.sleep(0.1)
                continue


    def main(self):
        ''' The main function
        
        This function is the main function of the game.
        It calls the play function and asks the user if they want to play again.
        If the user wants to play again,      the board is cleared and the game is played again.
        If the user wants to play a new game, the board is reset and the game is played again.
        If the user wants to exit,            the game is exited.
        '''

        while True:
            self.play()

            query = ''' Do you want to play again? :
            1. Quick restart (The board will be reset to the same dimensions)
            2. New game (You will be asked for the number of dimensions and the length of each dimension)
            3. Exit
            '''
            PlayAgain = self.input_handler_int(query, [1, 2, 3])
            if PlayAgain == 1:
                self.clear_the_board()
                continue
            elif PlayAgain == 2:
                self.create_new_board()
                continue
            elif PlayAgain == 3:
                break
            else:
                print("Something went wrong")
                print("Exiting the game")
                break


In [21]:
NDSudokuGame()

(2, 3, 1) :  1

(1, 3, 3) :  1

(3, 3, 3) :  2

(1, 2, 1) :  1

(3, 1, 1) :  1

(1, 1, 1) :  3

(2, 3, 3) :  3

(2, 3, 2) :  2

(2, 2, 2) :  1

(1, 3, 1) :  2

(2, 1, 1) :  2

(3, 3, 1) :  3

(3, 2, 2) :  3
The board is solvable!
Here is an example solution:
[[[3 1 2]
  [1 2 3]
  [2 3 1]]

 [[2 3 1]
  [3 1 2]
  [1 2 3]]

 [[1 2 3]
  [2 3 1]
  [3 1 2]]]

Game over!


In [13]:
B = nd_sudoku_Board(3, 3)
# B.set_cell((0, 0, 0, 0), 1)
# B.set_cell((0, 0, 0, 1), 2)
# B.set_cell((0, 0, 0, 2), 3)
# B.set_cell((0, 0, 0, 3), 4)
# B.set_cell((0, 0, 1, 3), 2)

# print(B.get_board())
possible, B_new, reason = Board_solver(B)
print(possible)
if possible:
    print(B_new)
print(reason)

True
[[[1 2 3]
  [2 3 1]
  [3 1 2]]

 [[2 3 1]
  [3 1 2]
  [1 2 3]]

 [[3 1 2]
  [1 2 3]
  [2 3 1]]]

